# OpenAI 适配器

对于部分已经适配 OpenAI 而尚未支持千帆 SDK 的第三库，本文提供了一种方法，可以快速将千帆适配至任意这类库。

千帆 SDK CLI 支持启动一个将千帆模拟成 OpenAI 接口的服务，然后让 OpenAI 请求该服务即可完成千帆的接入。

具体来说，首先需要在命令行中运行 `qianfan openai` 启动一个模拟服务。

可以通过 `-p 8001` 来指定端口，以及 `-d` 来让服务保持在后台运行。

以下是使用python代码启动子进程的方式运行：

In [ ]:
# import nest_asyncio
import os
import subprocess as sp

# 以下环境变量供千帆 OpenAI Adapter：
# 使用安全认证的access_key/secret_key:
os.environ["QIANFAN_ACCESS_KEY"] = "your_access_key"
os.environ["QIANFAN_SECRET_KEY"] = "your_secret_key"
# 或者应用ak/sk
# os.environ["QIANFAN_AK"]="your_app_ak"
# os.environ["QIANFAN_SK"]="your_app_sk"

# 启动千帆 SDK 的 OpenAI Adapter 服务，用于代理转发指向 OpenAI 的请求，并替换为千帆服务的返回
server = sp.Popen("qianfan openai -p 8001", shell=True)
# 也可以在终端直接执行qianfan openai -p 8001

当服务启动后，只需要设置两个环境变量，就可以在代码中直接调用支持 OpenAI 的第三方库，这里以 langchain 为例：

In [6]:
import os

os.environ["OPENAI_API_KEY"] = "any-content"  # 任意内容，仅为绕过 OpenAI SDK 验证
os.environ["OPENAI_BASE_URL"] = "http://127.0.0.1:8001/v1"  # 修改为本地 OpenAI API 地址

In [3]:
from langchain_openai import OpenAI
from langchain_core.messages import HumanMessage, SystemMessage

llm = OpenAI()

llm.invoke([
    HumanMessage(
        content="你是谁"
    ),
])

'你好，我是百度研发的文心一言。我可以回答你的问题或与你对话。请问你有什么具体的问题或需要帮助吗？'

也支持使用 function_call 实现 Agent

In [5]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchRun

In [3]:
tools = [DuckDuckGoSearchRun()]
prompt = hub.pull("hwchase17/openai-tools-agent")
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)

# Construct the OpenAI Tools agent
agent = create_openai_tools_agent(llm, tools, prompt)

In [11]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input": "请通过搜索告诉我文心千帆是什么"})



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_search` with `{'query': '文心千帆'}`


文心千帆是由百度推出的一款AI工具，它集合了PPT制作、数字人主播一键开播等数十种应用场景，致力于帮助用户提升数字内容创作效率。 文心千帆采用了先进的AI技术，可以根据用户需求自动生成高质量的PPT、 视频 等内容，让创作变得更加轻松和高效。 据介绍，「文心一言」目前包含以下五类落地场景：文学创作、商业文案创作、数理逻辑推送、中文理解、多模态生成。 文心一言是百度基于文心大模型技术推出的生成式对话产品。 什么是千帆大模型平台？ 百度智能云千帆大模型平台（以下简称千帆或千帆大模型平台）是面向企业开发者的一站式大模型开发及服务运行平台。 千帆不仅提供了包括文心一言底层模型（ERNIE-Bot）和第三方开源大模型，还提供了各种AI开发工具和整套开发环境，方便客户轻松使用和开发大模型应用。 支持数据管理、自动化模型SFT以及推理服务云端部署的一站式大模型定制服务，助力各行业的生成式AI应用需求落地。 高性能：百度文心千帆 大模型平台采用了先进的深度学习框架和高效的计算资源，确保模型推理的速度和准确性。 三、使用体验 在使用百度文心千帆大模型平台的过程中，我主要体验了文本分类和情感分析两个功能。下面我将通过实例来展示这 ... 什么是百度智能云千帆大模型平台. 更新时间 ： 2024-03-08. 目录. 视频介绍. 产品定义. 应用场景. 上线啦!. 与广大开发者共同探索大模型世界的无限可能! 关于应用程序如何备案， 点击进入 来看看吧～. 文心千帆在开发过程中充分运用了RLHF技术，通过收集用户的反馈数据来优化模型的表现。这种技术使得模型能够不断学习和改进，以提供更加符合用户需求的服务。 四、实际应用与建议 文心千帆的PPT制作和数字人直播功能在实际应用中取得了显著的效果。文心千帆是由百度推出的一款AI工具，它集合了PPT制作、数字人主播一键开播等数十种应用场景，致力于帮助用户提升数字内容创作效率。文心千帆采用了先进的AI技术，可以根据用户需求自动生成高质量的PPT、视频等内容，让创作变得更加轻松和高效。同时，文心千帆还提供了包括文心一言底层模型（ERNIE-Bot）和第三方开源

{'input': '请通过搜索告诉我文心千帆是什么',
 'output': '文心千帆是由百度推出的一款AI工具，它集合了PPT制作、数字人主播一键开播等数十种应用场景，致力于帮助用户提升数字内容创作效率。文心千帆采用了先进的AI技术，可以根据用户需求自动生成高质量的PPT、视频等内容，让创作变得更加轻松和高效。同时，文心千帆还提供了包括文心一言底层模型（ERNIE-Bot）和第三方开源大模型在内的各种AI开发工具和整套开发环境，方便客户轻松使用和开发大模型应用。文心千帆在开发过程中充分运用了RLHF技术，通过收集用户的反馈数据来优化模型的表现，使得模型能够不断学习和改进，以提供更加符合用户需求的服务。'}